# Model v3 (ours)

In [ ]:
from keras.layers.core import Dense, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten, AvgPool2D
from keras.layers import BatchNormalization, Dropout
from keras.layers.convolutional import DepthwiseConv2D
from keras.backend import relu
from keras.activations import softmax
from keras import regularizers

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import cv2

data_dir = Path('/home/gleb/programming/play/tiny_ml/mini_face_dataset')
batch_size = 32
img_height = 48
img_width = 48

train_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_dir / 'train'),
  seed=0,
  color_mode='grayscale',
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_dir / 'validation'),
  seed=0,
  color_mode='grayscale',
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)

num_classes = len(class_names)
dict_ = {}
for i in range(num_classes):
    dict_[i] = class_names[i]

model_path = './saved_model_weights_v3/best_model'

model = Sequential()

model.add(DepthwiseConv2D((3,3),input_shape=(img_width,img_height,1)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))

model.add(Conv2D(2, (3, 3)))
model.add(BatchNormalization(axis=-1))
convLayer01 = Activation('relu')

model.add(Conv2D(2, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(convLayer01)
convLayer015 = AvgPool2D(pool_size=(2,2))
model.add(convLayer015)

model.add(Conv2D(4, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))

model.add(Conv2D(4, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
convLayer02 = AvgPool2D(pool_size=(2,2))
model.add(convLayer02)

model.add(Conv2D(8,(3, 3)))
model.add(BatchNormalization(axis=-1))
convLayer03 = Activation('relu')

model.add(Conv2D(8,(3, 3)))
model.add(BatchNormalization(axis=-1))
convLayer03 = Activation('relu')
convLayer04 = AvgPool2D(pool_size=(2,2))
model.add(convLayer04)

model.add(Flatten())                                 
model.add(Dense(5, activation = relu, kernel_regularizer=regularizers.L2(0.05))) 
model.add(Dense(num_classes, activation = softmax))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.build((None, img_height, img_height, 3))
model.summary()

In [ ]:
model.load_weights(model_path)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  predictions = model.predict(images)
  max_predictions = np.argmax(predictions, axis=1)
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(f'{dict_[max_predictions[i]]} ({class_names[labels[i]]})\n{predictions[i][0]:.2f}, {predictions[i][1]:.2f}, {predictions[i][2]:.2f}')
    plt.axis("off")

In [ ]:
#input_ = cv2.imread('ghappy.png', 0)
input_ = cv2.imread('gfrown.png', 0)
input_ = cv2.imread('output/14.png', 0)
#input_ = np.zeros((48, 48), dtype=np.uint8)
input_ = cv2.resize(input_, (48, 48))
plt.imshow(input_.astype(np.uint8))
print(input_.shape)
input_ = np.expand_dims(input_, axis=0)
print(input_.shape)
input_tf = tf.convert_to_tensor(input_, dtype=tf.float32)
pred = model.predict([input_tf])
print(pred*255)
print(dict_[np.argmax(pred, axis=1)[0]])
#help(tf.convert_to_tensor)
print(images[0].shape)
with open('foo.txt', 'w') as fid:
    for i in range(input_.shape[1]):
        for j in range(input_.shape[2]):
            fid.write(f'{input_[0][i, j]}, ')
input_.shape

# Saving the model

In [ ]:
!apt-get -qq install xxd

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
# Save the model to disk
open("face_model_prax.tflite", "wb").write(tflite_model)

In [ ]:
import os
os.path.getsize('face_model_prax.tflite')

!echo "const unsigned char model[] = {" > model.h
!cat face_model_v3.tflite | xxd -i      >> model.h
!echo "};"                              >> model.h